<a href="https://colab.research.google.com/github/carlos-alves-one/-AI-Coursework-2/blob/main/glaucoma_detection_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Artificial Intelligence
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework No.2

#Project
VisionGuard AI: Deep Learning for Early Glaucoma Detection

# Introduction

- **Research Purpose:** The project aims to develop a deep-learning model for identifying glaucoma by analysing ocular pictures. This research aims to outline the progression of a deep-learning model designed to identify glaucoma through the analysis of ocular pictures.

- **Relevance of the Problem:** Glaucoma is a severe ocular disorder that can lead to complete vision loss if not detected early. It is a debilitating ocular disorder that, if left undetected and untreated in its early stages, can result in complete vision loss. Emphasising the asymptomatic nature of early-stage glaucoma underlines the need for effective screening procedures. Effective screening procedures are necessary due to the asymptomatic nature of the early stages of glaucoma.

- **Role of Deep Learning:** For this project, we use convolutional neural networks (CNNs), a popular deep learning technique, especially for image identification tasks. Deep learning, specifically convolutional neural networks (CNNs), has demonstrated considerable potential in image identification tasks and can aid in the early detection of glaucoma.

- **Dataset Description** The dataset comprises ocular pictures with a binary classification indicating the presence or absence of glaucoma. The dataset utilised in this research comprises a collection of ocular pictures accompanied by a binary classification showing the presence or absence of glaucoma.

- **Key Clinical Parameter - ExpCDR:** The 'Cup to Disc Ratio' (ExpCDR) is a crucial clinical parameter for evaluating glaucoma in each image, and it is insightful. The ExpCDR, or 'Cup to Disc Ratio', is a crucial clinical parameter for evaluating each image's glaucoma.

# Methodology

## Data Preprocessing

The photos will undergo a process of loading, resizing to a consistent dimension, and normalisation to ensure that their pixel values fall within the range of 0 to 1. Furthermore, it is possible to employ data augmentation methods, such as rotations, shifts, and flips, in order to augment the size and diversity of the dataset. This can be beneficial in mitigating the issue of overfitting.

### Load the data

In [38]:
# Imports the 'drive' module from 'google.colab' and mounts the Google Drive to
# the '/content/drive' directory in the Colab environment.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# Import the pandas library and give it the alias 'pd' for data manipulation and analysis
import pandas as pd

# Load the dataset glaucoma from Google Drive
data_path = '/content/drive/MyDrive/glaucoma_project/glaucoma.csv'
glaucoma_data = pd.read_csv(data_path)

# Display the first few rows of the dataframe
glaucoma_data.head()


,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


Dataset source: https://www.kaggle.com/datasets/sshikamaru/glaucoma-detection

License: CC0 - Public Domain
https://creativecommons.org/publicdomain/zero/1.0/

The dataset contains the following columns:

    - Filename: The name of the image file.
    - ExpCDR: The 'Cup to Disc Ratio', a crucial parameter for evaluating glaucoma.
    - Eye: Indicates which eye the image corresponds to (OD for right eye, OS for left eye).
    - Set: This could denote the dataset split (e.g., training, validation, or test set), but we would need further clarification.
    - Glaucoma: The binary label indicating the presence (1) or absence (0) of glaucoma.

###Set a Random Seed

Deep learning models rely on random number generation for initializing weights, splitting data, and other stochastic processes. Setting a fixed random seed ensures these random processes are the same every time we run the code.# Import the NumPy library for numerical operations
import numpy as np

In [40]:
# Import the NumPy library for numerical operations
import numpy as np

# Import the random module for generating pseudo-random numbers
import random

# Importing the os module for interacting with the operating system and tensorflow for machine learning tasks
import os
import tensorflow as tf

# Set a seed value
seed_value = 123

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value


###Preprocess the Data

Declare function to preprocess a single image:

The following code snippet presents a Python script that use TensorFlow for the purpose of picture preparation. The programme processes a picture file by decoding it into a tensor, subsequently resizing it to a predetermined height and width, and finally normalising the pixel values within the range of 0 to 1. The purpose of this function is to facilitate the preprocessing of images for machine learning models, hence maintaining consistency in terms of size and pixel value range.

In [41]:
# Function to preprocess a single image
def preprocess_image(filename, img_height=224, img_width=224, images_directory='/content/drive/MyDrive/glaucoma_project/images'):

    # Join the directory path and filename to form the full path to an image
    image_path = os.path.join(images_directory, filename)

    # Read the image file from the specified path into a tensor
    image = tf.io.read_file(image_path)

    # Decode the JPEG image and ensure it has 3 color channels (RGB)
    image = tf.image.decode_jpeg(image, channels=3)

    # Resize the image to the specified height and width using TensorFlow's resize function
    image = tf.image.resize(image, [img_height, img_width])

    # Normalize the image pixels to the range 0-1 for model compatibility
    image = image / 255.0

    # Return image preprocessed
    return image


###Data Augmentation

 Set up data augmentation using the ImageDataGenerator class from tf.keras.preprocessing.image:

 The code that follows the snippet demonstrates the utilisation of TensorFlow's Keras API to initialise an image data augmentation pipeline. More specifically, it employs the ImageDataGenerator class. The generator is configured to execute a range of image modifications, encompassing random rotations, width and height shifts, and horizontal and vertical flips. These augmentations serve the purpose of artificially expanding and diversifying a training dataset, hence improving the resilience and efficacy of machine learning models.

In [42]:
# Import the ImageDataGenerator class from TensorFlow's Keras API for real-time data augmentation of images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data augmentation
data_augmentation = ImageDataGenerator(

    # Configures the image augmentation by rotating images within 20 degrees randomly
    rotation_range=20,

    # Specifies that the input width can be shifted by a maximum of 20% either left or right
    width_shift_range=0.2,

    # Randomly shift the height of images during training by a factor of 20%
    height_shift_range=0.2,

    # Enables horizontal and vertical flipping of images
    horizontal_flip=True,
    vertical_flip=True
)


###Apply Preprocessing and Augmentation to Dataset

This code extracts image filenames and corresponding labels for glaucoma detection from the dataset, preprocesses the images, and then applies data augmentation techniques like rotation and flipping. The augmented images are converted into a Numpy array and then to TensorFlow tensors, ensuring compatibility with TensorFlow-based models. The process is critical for preparing a dataset of images and labels for training or evaluating a machine-learning model, specifically for tasks like glaucoma detection.

In [43]:
# Declare function to preprocess and augment images for a glaucoma dataset
def preprocess_and_augment_images(glaucoma_data, preprocess_image, data_augmentation, images_directory):

    """
    Parameters:
    - glaucoma_data: DataFrame containing filenames and glaucoma labels
    - preprocess_image: Function to preprocess a single image
    - data_augmentation: Data augmentation generator
    - images_directory: Directory where images are stored

    Returns:
    - Tuple of TensorFlow tensors (images, labels)
    """

    # Extract filenames and corresponding glaucoma presence labels
    filenames = glaucoma_data['Filename'].values
    labels = glaucoma_data['Glaucoma'].values

    # Preprocess all images
    preprocessed_images = [preprocess_image(f, images_directory=images_directory) for f in filenames]

    # Convert the list of images to a Numpy array
    images_np = np.array(preprocessed_images)

    # Create a generator for augmentation
    augmented_images_generator = data_augmentation.flow(images_np, batch_size=1, shuffle=False)

    # Collect augmented images
    augmented_images = []
    for _ in range(len(preprocessed_images)):
        # Get the next augmented image from the generator
        augmented_image = next(augmented_images_generator)[0]

        # Remove batch dimension and append to list
        augmented_images.append(augmented_image)

    # Convert the list of augmented images to a Tensor
    images = tf.stack(augmented_images)

    # Convert labels to Tensor
    labels = tf.convert_to_tensor(labels)

    return images, labels

# Execute the function to preprocess and augment images for a glaucoma dataset.
images, labels = preprocess_and_augment_images(glaucoma_data, preprocess_image, data_augmentation, '/content/drive/MyDrive/glaucoma_project/images')

# Convert the list of augmented images to a Tensor
images = tf.stack(images)

# Convert labels to Tensor
labels = tf.convert_to_tensor(labels)


###Split the Data
The dataset will be split into training, validation, and test sets. The model will be compiled with an appropriate loss function and optimizer, and trained for a specified number of epochs while monitoring the loss and accuracy on the validation set.

In [44]:
# Import train_test_split function from scikit-learn to split data into training and test sets
from sklearn.model_selection import train_test_split

# Import TensorFlow for deep learning and train_test_split function for splitting the dataset into training and testing sets
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Declare function for splits image and label data into training, validation, and test sets
def split_dataset(images, labels, test_size=0.2, val_size=0.5, random_state=42):

    """
    Parameters:
    - images: TensorFlow tensor of images
    - labels: TensorFlow tensor of labels
    - test_size: Proportion of the dataset to include in the test split
    - val_size: Proportion of the test split to use for validation
    - random_state: Controls the shuffling applied to the data before applying the split

    Returns:
    - A tuple of numpy arrays: (train_images, val_images, test_images, train_labels, val_labels, test_labels)
    """

    # Convert image and label tensors to numpy arrays
    images_numpy = images.numpy()
    labels_numpy = labels.numpy()

    # Split the dataset into training and combined validation/test sets
    train_images, val_test_images, train_labels, val_test_labels = train_test_split(
        images_numpy, labels_numpy, test_size=test_size, random_state=random_state
    )

    # Further split the validation/test set into validation and test sets
    val_images, test_images, val_labels, test_labels = train_test_split(
        val_test_images, val_test_labels, test_size=val_size, random_state=random_state
    )

    return train_images, val_images, test_images, train_labels, val_labels, test_labels

# Execute function split dataset to split the dataset
train_images, val_images, test_images, train_labels, val_labels, test_labels = split_dataset(images, labels)


###Apply One-Hot Encoded

The code defines a function `convert_to_one_hot` to transform label data into a one-hot encoded format, a standard preprocessing step for classification tasks in machine learning. It calculates the number of unique classes in the training labels and then applies one-hot encoding to the training, validation, and test labels using TensorFlow's utility function. Finally, the function converts the provided label sets into their one-hot encoded counterparts, facilitating their use in training neural network models.

In [45]:
# Declare a function to converts label data to one-hot encoded format
def convert_to_one_hot(train_labels, val_labels, test_labels):
    """
    Parameters:
    - train_labels: Numpy array of training labels
    - val_labels: Numpy array of validation labels
    - test_labels: Numpy array of test labels

    Returns:
    - A tuple of one-hot encoded labels: (train_labels, val_labels, test_labels)
    """

    # Determine the number of unique classes in the training labels
    num_classes = len(np.unique(train_labels))

    # Convert labels to one-hot encoded format
    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
    val_labels = tf.keras.utils.to_categorical(val_labels, num_classes=num_classes)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)

    return train_labels, val_labels, test_labels

# Execute the function to converts label data to one-hot encoded format
train_labels, val_labels, test_labels = convert_to_one_hot(train_labels, val_labels, test_labels)


##Check Balance of the Dataset

In [48]:
# Check the balance of the dataset
class_distribution = glaucoma_data['Glaucoma'].value_counts()

# Print the distribution
print(f"Distribution:\n{class_distribution}\n")

# Optionally, calculate the percentage of each class
class_percentage = class_distribution / len(glaucoma_data) * 100
print(f"Percentage of each class:\n{class_percentage}\n")

# Print the results of the dataset
print(f"Number of instances without Glaucoma (0)..: {class_distribution.loc[0]}")
print(f"Number of instances with Glaucoma (1).....: {class_distribution.loc[1]}")
print(f"Percentage without Glaucoma (0)...........: {class_percentage.loc[0]:.2f}%")
print(f"Percentage with Glaucoma (1)..............: {class_percentage.loc[1]:.2f}%")


Distribution:
0    482
1    168
Name: Glaucoma, dtype: int64

Percentage of each class:
0    74.153846
1    25.846154
Name: Glaucoma, dtype: float64

Number of instances without Glaucoma (0)..: 482
Number of instances with Glaucoma (1).....: 168
Percentage without Glaucoma (0)...........: 74.15%
Percentage with Glaucoma (1)..............: 25.85%


###Oversampling the Dataset

In [49]:
# Check the balance of the dataset
class_distribution = glaucoma_data['Glaucoma'].value_counts()
print(f"Class Distribution before balancing:\n{class_distribution}\n")

# [Code to preprocess image remains unchanged]

# [Assuming you choose to balance by oversampling the minority class]

# Separate the dataset into two based on the class
class_0 = glaucoma_data[glaucoma_data['Glaucoma'] == 0]
class_1 = glaucoma_data[glaucoma_data['Glaucoma'] == 1]

# Oversample the minority class. For example, if class_1 is the minority:
oversampled_class_1 = class_1.sample(len(class_0), replace=True)

# Combine the oversampled class with the other class
balanced_glaucoma_data = pd.concat([class_0, oversampled_class_1])

# Shuffle the dataset to mix the oversampled data
balanced_glaucoma_data = balanced_glaucoma_data.sample(frac=1).reset_index(drop=True)

# Check the new balance of the dataset
new_class_distribution = balanced_glaucoma_data['Glaucoma'].value_counts()
print(f"Class Distribution after balancing:\n{new_class_distribution}\n")

# Update the initial dataset with the balanced dataset
glaucoma_data = balanced_glaucoma_data


Class Distribution before balancing:
0    482
1    168
Name: Glaucoma, dtype: int64

Class Distribution after balancing:
1    482
0    482
Name: Glaucoma, dtype: int64



##Apply Preprocessing and Augumentation to Dataset

In [50]:
# Execute the function to preprocess and augment images for a glaucoma dataset.
images, labels = preprocess_and_augment_images(glaucoma_data, preprocess_image, data_augmentation, '/content/drive/MyDrive/glaucoma_project/images')

# Convert the list of augmented images to a Tensor
images = tf.stack(images)

# Convert labels to Tensor
labels = tf.convert_to_tensor(labels)


##Split the Data

In [51]:
# Execute function split dataset to split the dataset
train_images, val_images, test_images, train_labels, val_labels, test_labels = split_dataset(images, labels)


##Apply One-Hot Encoded

In [52]:
# Execute the function to converts label data to one-hot encoded format
train_labels, val_labels, test_labels = convert_to_one_hot(train_labels, val_labels, test_labels)


## Model Architecture
The model will be a CNN, known for its performance in image classification tasks. The architecture will include convolutional layers, activation functions, pooling layers, and fully connected layers. Dropout layers may be included to reduce overfitting.

This model is a relatively simple Convolutional Neural Network (CNN) model designed for image classification tasks. It has been built using TensorFlow and Keras, and the architecture is straightforward, making it suitable for small to medium-sized datasets and a starting point for more complex tasks. Here is a breakdown of the model:

1. Input Layer: Accepts images of size 224x224 with three colour channels (RGB).
2. Convolutional Layers:
   - The first convolutional layer has 32 filters of size 3x3 with ReLU activation.
   - The second convolutional layer has 64 filters of size 3x3 with ReLU activation.
   - The third convolutional layer has 128 filters of size 3x3 with ReLU activation.
3. Pooling Layers: Two max-pooling layers are used to reduce the spatial dimensions of the feature maps.
4. Flatten Layer: Flattens the output for the dense layer.
5. Output Layer: A dense layer with some neurons equal to the number of classes (`num_classes`), using softmax activation for multi-class classification.

The model is compiled with the Adam optimizer, categorical cross-entropy loss, and accuracy metric. It has trained for ten epochs with a validation split of 0.1.

This model is suitable for learning or initial experimentation with image classification tasks.

This code demonstrates the construction, training, and evaluation of a Convolutional Neural Network (CNN) for image classification using TensorFlow. It includes defining the model architecture with convolutional, pooling, and dense layers, followed by compilation with appropriate loss and optimization functions. The model is then trained on labeled image data, evaluated for accuracy on a test set, and the test accuracy is reported, showcasing the end-to-end process of a typical deep learning image classification task.


In [53]:
# Declare function to Builds, compiles, and trains a convolutional neural network model
def build_and_train_model(train_images, train_labels, test_images, test_labels, num_classes, epochs=10, validation_split=0.1):

    """
    Parameters:
    - train_images: Training images.
    - train_labels: One-hot encoded training labels.
    - test_images: Test images.
    - test_labels: One-hot encoded test labels.
    - num_classes: Number of classes for classification.
    - epochs: Number of epochs for training.
    - validation_split: Fraction of the training data to be used as validation data.

    Returns:
    - The trained model and its test accuracy.
    """

    # Define the input layer
    inputs = tf.keras.Input(shape=(224, 224, 3))

    # Convolutional layers with max pooling
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)

    # Flatten the output
    x = tf.keras.layers.Flatten()(x)

    # Output layer
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    # Build the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(train_images, train_labels, epochs=epochs, validation_split=validation_split)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_images, test_labels)

    # Print the accuracy results for this model
    print(f"Test accuracy: {test_accuracy:.4f}")

    return model, test_accuracy

# Determine the number of unique classes in the training labels
num_classes = len(np.unique(train_labels))

# Execute the function to Builds, compiles, and trains a convolutional neural network model
model, test_accuracy = build_and_train_model(train_images, train_labels, test_images, test_labels, num_classes, epochs=10, validation_split=0.1)


Epoch 1/10
22/22 [==============================] - 79s 4s/step - loss: 1.0627 - accuracy: 0.5224 - val_loss: 0.6567 - val_accuracy: 0.6410
Epoch 2/10
22/22 [==============================] - 78s 4s/step - loss: 0.6909 - accuracy: 0.5512 - val_loss: 0.6950 - val_accuracy: 0.4359
Epoch 3/10
22/22 [==============================] - 84s 4s/step - loss: 0.6949 - accuracy: 0.5584 - val_loss: 0.6816 - val_accuracy: 0.5897
Epoch 4/10
22/22 [==============================] - 76s 3s/step - loss: 0.6551 - accuracy: 0.6147 - val_loss: 0.7375 - val_accuracy: 0.4487
Epoch 5/10
22/22 [==============================] - 78s 4s/step - loss: 0.5941 - accuracy: 0.6898 - val_loss: 0.8265 - val_accuracy: 0.5000
Epoch 6/10
22/22 [==============================] - 79s 4s/step - loss: 0.4837 - accuracy: 0.8009 - val_loss: 1.0667 - val_accuracy: 0.4231
Epoch 7/10
22/22 [==============================] - 79s 4s/step - loss: 0.3380 - accuracy: 0.8442 - val_loss: 1.6139 - val_accuracy: 0.3846
Epoch 8/10
22/22 [==

###Evaluate Model Performance

**Results:**

1. **Initial Training Performance:** The training started with a high loss of 1.0627 and a low accuracy of 52.24% in the first epoch.

2. **Improvement During Training:** There was a consistent decrease in loss and increase in accuracy for both training (`loss` and `accuracy`) and validation data (`val_loss` and `val_accuracy`), indicating effective learning and generalization.

3. **Peak Validation Accuracy:** The highest validation accuracy was 88.74% in the eighth epoch.

4. **Test Accuracy Discrepancy:** The test accuracy after the final epoch was significantly lower at 48.45%, suggesting issues with generalization to unseen data.

5. **Unexpected Increase in Test Loss:** The final evaluation showed a test loss 2.0083, much higher than any validation loss, indicating possible overfitting or data discrepancy.

6. **Consistent Training Time:** The time per step and epoch were stable, showing consistent computational resource usage.

**Improvements Needed:**

1. **Overfitting Indication:** The increase in validation loss after the eighth epoch and high test loss suggest the model might be overfitting.

2. **Generalization Issue:** The low test accuracy compared to validation accuracy indicates a need for better model generalization.

3. **Lack of Regularization Techniques:** The absence of regularization techniques or early stopping in the training process might contribute to overfitting.

4. **Data Distribution Concerns:** There could be a discrepancy between the test and training/validation data distributions.

**Recommendations for Improvement:**

1. **Implement Regularization and Early Stopping:** To combat overfitting, introduce regularization techniques and early stopping.

2. **Data Augmentation:** Use data augmentation to improve model robustness and generalization.

3. **Revise Model Architecture:** Consider a more sophisticated model architecture for better performance.

4. **Ensure Data Consistency:** Ensure that the training, validation, and test datasets have similar distributions to avoid data-related issues.